# Interactive Surface Codes with `deltakit` and `crumpy`

In this notebook, we show how `deltakit` can be used to generate surface code memory experiments and various ways in which visualization packages like `crumpy` and `plotly` can be used to create interactive interfaces for surface code exploration. We show examples of generating planar codes with `deltakit.explorer.codes`, how to interactively visualize surface code experiment circuits with `crumpy` and `plotly`, and how to add Pauli error markers to those generated circuits to explore the error-detecting capabilities of surface codes.

### Required Imports

In [1]:
from deltakit.explorer.codes import RotatedPlanarCode, UnrotatedPlanarCode, css_code_memory_circuit
from deltakit.circuit import GateLayer, Circuit
from deltakit.circuit.gates import PauliBasis
from crumpy import CircuitWidget
from snippet import draw_patch, update_patch
from plotly.callbacks import Points

import ipywidgets
import stim
import traitlets

## Using `deltakit` with `crumpy`

To start, let's generate and visualize a 2 x 2 `RotatedPlanarCode` with `deltakit.explorer.codes`. The following will output a static image of a 2 x 2 rotated planar code patch (we'll come back to a similar visualization with more interactivity later):

In [2]:
# Generate a deltakit planar code with the given width and height
def getPlanarCode(is_rotated: bool, w: int, h: int) -> RotatedPlanarCode | UnrotatedPlanarCode:
    return RotatedPlanarCode(width=w, height=h) if is_rotated else UnrotatedPlanarCode(width=w, height=h)

# Creates a visual representation of the given planar code, saving it to ./images/ and returning a viewable widget
def savePlanarCodeImage(code: RotatedPlanarCode | UnrotatedPlanarCode) -> ipywidgets.Image:
    filename = f"./images/{'u' if isinstance(code, UnrotatedPlanarCode) else ''}rpc_{code.width}x{code.height}.png"
    code.draw_patch(filename)

    file = open(filename, "rb")
    image = file.read()
    image_widg = ipywidgets.Image(
        value=image,
        format="png",
        width=500
    )
    return image_widg

code = getPlanarCode(is_rotated=True, w=2, h=2)
img = savePlanarCodeImage(code)
img

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\xe0\x08\x06\x00\x00\x005\xd1\xdc…

With our 2 x 2 `RotatedPlanarCode`, we can create a memory experiment circuit using `css_code_memory_circuit`:

In [3]:
deltakit_circuit = css_code_memory_circuit(code, num_rounds=1, logical_basis=PauliBasis.Z)
deltakit_circuit

Circuit([
    GateLayer([
        RZ(Qubit(Coord2D(3, 1)))
        RZ(Qubit(Coord2D(1, 1)))
        RZ(Qubit(Coord2D(3, 3)))
        RZ(Qubit(Coord2D(1, 3)))
        RX(Qubit(Coord2D(2, 0)))
        RX(Qubit(Coord2D(2, 2)))
        RX(Qubit(Coord2D(2, 4)))
    ])
    GateLayer([
        I(Qubit(Coord2D(1, 1)))
        I(Qubit(Coord2D(1, 3)))
        I(Qubit(Coord2D(3, 1)))
        I(Qubit(Coord2D(3, 3)))
    ])
    GateLayer([
        CZ(control=Qubit(Coord2D(2, 0)), target=Qubit(Coord2D(1, 1)))
        CX(control=Qubit(Coord2D(2, 2)), target=Qubit(Coord2D(1, 3)))
    ])
    GateLayer([
        CZ(control=Qubit(Coord2D(2, 0)), target=Qubit(Coord2D(3, 1)))
        CX(control=Qubit(Coord2D(2, 2)), target=Qubit(Coord2D(1, 1)))
    ])
    GateLayer([
        CX(control=Qubit(Coord2D(2, 2)), target=Qubit(Coord2D(3, 3)))
        CZ(control=Qubit(Coord2D(2, 4)), target=Qubit(Coord2D(1, 3)))
    ])
    GateLayer([
        CX(control=Qubit(Coord2D(2, 2)), target=Qubit(Coord2D(3, 1)))
        CZ

To visualize this circuit with `crumpy`'s `CircuitWidget`, we first convert our `deltakit` circuit to a `stim` circuit:

In [4]:
stim_circuit = deltakit_circuit.as_stim_circuit()
stim_circuit

stim.Circuit('''
    QUBIT_COORDS(1, 3) 0
    QUBIT_COORDS(2, 4) 1
    QUBIT_COORDS(3, 3) 2
    QUBIT_COORDS(2, 2) 3
    QUBIT_COORDS(3, 1) 4
    QUBIT_COORDS(1, 1) 5
    QUBIT_COORDS(2, 0) 6
    R 4 5 2 0
    RX 6 3 1
    TICK
    I 5 0 4 2
    TICK
    CZ 6 5
    CX 3 0
    TICK
    CZ 6 4
    CX 3 5
    TICK
    CX 3 2
    CZ 1 0
    TICK
    CX 3 4
    CZ 1 2
    TICK
    MX 6 3 1
    M 4 5 2 0
    DETECTOR(2, 0, 0) rec[-7]
    DETECTOR(2, 4, 0) rec[-5]
    SHIFT_COORDS(0, 0, 1)
    OBSERVABLE_INCLUDE(0) rec[-3] rec[-1]
    DETECTOR(2, 0, 0) rec[-7] rec[-4] rec[-3]
    DETECTOR(2, 4, 0) rec[-5] rec[-1] rec[-2]
    SHIFT_COORDS(0, 0, 1)
''')

`stim.Circuit`s can be displayed directly with `CircuitWidget.from_stim`:

In [5]:
CircuitWidget.from_stim(stim_circuit)

CircuitWidget(stim='QUBIT_COORDS(1, 3) 0\nQUBIT_COORDS(2, 4) 1\nQUBIT_COORDS(3, 3) 2\nQUBIT_COORDS(2, 2) 3\nQU…

Let's streamline this process of generating `stim` from a planar code specification. `stimSurfaceCode` lets us generate a (`crumpy`-compatible) `stim` circuit for a rotated/unrotated planar code with a given `w` and `h`. Note that we also strip away any non-gate layers just to reduce visual clutter on the circuit diagrams:

In [6]:
# Generates a stim.Circuit representing a memory experiment on a planar code
def stimSurfaceCode(is_rotated: bool, w: int, h: int) -> stim.Circuit:
    code = getPlanarCode(is_rotated, w, h)
    circuit = css_code_memory_circuit(code, num_rounds=1, logical_basis=PauliBasis.Z)
    circuit = Circuit([
            layer for layer in circuit.layers 
            if isinstance(layer, GateLayer)
        ])
    return circuit.as_stim_circuit()

stim_circuit = stimSurfaceCode(True, 3, 3)
CircuitWidget.from_stim(stim_circuit)

CircuitWidget(stim='QUBIT_COORDS(2, 4) 0\nQUBIT_COORDS(4, 0) 1\nQUBIT_COORDS(1, 5) 2\nQUBIT_COORDS(3, 1) 3\nQU…

Now that we have a way to easily generate the `stim` circuit for a planar code memory experiment, we can make things more interactive with some control widgets (from `ipywidgets`). Let's add some sliders that vary the width and height of the planar code we use, and a checkbox for `UnrotatedPlanarCode` or `RotatedPlanarCode`:

In [7]:
# An ipywidgets.IntSlider with with '+' and '-' buttons at the ends
class SliderWithButtons(ipywidgets.Box):
    value = traitlets.Int()

    def __init__(self, description: str="Value", value: int=0, min: int=0, max: int=10, step: int=1, orientation: str="horizontal", readout: bool=True):
        # Internal widgets
        self.slider = ipywidgets.IntSlider(
            description=description,
            value=value,
            min=min,
            max=max,
            step=step,
            orientation=orientation,
            readout=readout,
            style={"description_width": "60px"}
        )
        self.minus = ipywidgets.Button(description="–", layout=ipywidgets.Layout(width="30px"))
        self.plus = ipywidgets.Button(description="+", layout=ipywidgets.Layout(width="30px"))

        # Arrange children
        if orientation == "vertical":
            container = ipywidgets.VBox([self.plus, self.slider, self.minus], layout=ipywidgets.Layout(align_items="center"))
        else:
            container = ipywidgets.HBox([self.minus, self.slider, self.plus])
        super().__init__(children=[container])

        # Link .value trait
        ipywidgets.link((self.slider, "value"), (self, "value"))

        # Button click events
        self.plus.on_click(self._on_plus)
        self.minus.on_click(self._on_minus)

    def _on_plus(self, _):
        if self.slider.value < self.slider.max:
            self.slider.value += self.slider.step

    def _on_minus(self, _):
        if self.slider.value > self.slider.min:
            self.slider.value -= self.slider.step

# Helpers for creating planar code config widgets (these will be reused later)
def WSlider():
    return SliderWithButtons(description="Width", value=3, min=2, max=5)
def HSlider():
    return SliderWithButtons(description="Height", value=3, min=2, max=5)
def Checkbox():
    return ipywidgets.Checkbox(
        value=True,
        description="Rotated?",
        disabled=False,
    )

# Create planar code config widgets + circuit widget
w_slider = WSlider()
h_slider = HSlider()
checkbox = Checkbox()
dynamic_circuit = CircuitWidget()

# Link config widgets to the circuit
dynamic_circuit_callback = lambda _: str(stimSurfaceCode(checkbox.value, w_slider.value, h_slider.value))
ipywidgets.dlink((w_slider, "value"), (dynamic_circuit, "stim"), dynamic_circuit_callback)
ipywidgets.dlink((h_slider, "value"), (dynamic_circuit, "stim"), dynamic_circuit_callback)
ipywidgets.dlink((checkbox, "value"), (dynamic_circuit, "stim"), dynamic_circuit_callback)

display(ipywidgets.Label("Planar code settings:"), checkbox, w_slider, h_slider, dynamic_circuit)

Label(value='Planar code settings:')

Checkbox(value=True, description='Rotated?')

SliderWithButtons(children=(HBox(children=(Button(description='–', layout=Layout(width='30px'), style=ButtonSt…

SliderWithButtons(children=(HBox(children=(Button(description='–', layout=Layout(width='30px'), style=ButtonSt…

CircuitWidget(stim='QUBIT_COORDS(2, 4) 0\nQUBIT_COORDS(4, 0) 1\nQUBIT_COORDS(1, 5) 2\nQUBIT_COORDS(3, 1) 3\nQU…

## Pauli Propagation

Using our `stimSurfaceCode` circuit generation function, let's explore another feature of `crumpy` and add a Pauli marker to a circuit:

In [8]:
# Prepend a Pauli X marker on qubit 0 (based on qubit coords, idx 0 is (1, 3))
stim_str_with_pauli = "#!pragma MARKX(0) 0\n" + str(stimSurfaceCode(True, 2, 2))

CircuitWidget(stim=stim_str_with_pauli)

CircuitWidget(stim='#!pragma MARKX(0) 0\nQUBIT_COORDS(1, 3) 0\nQUBIT_COORDS(2, 4) 1\nQUBIT_COORDS(3, 3) 2\nQUB…

We can add some interactivity with a slider (to change which qubit line the Pauli marker/error is on) and some radio buttons (to select the Pauli type):

In [9]:
# Error slider and radio buttons
def PauliSlider():
    return SliderWithButtons(description="", value=3, min=0, orientation="vertical", readout=False)
def PauliSelect():
    return ipywidgets.RadioButtons(
        options=["X", "Y", "Z", "none"],
        default="X",
        description="Pauli error:",
        disabled=False
    )

# Returns a stim circuit with a specified Pauli err marker added at the front
# (on the idx-th circuit line from bottom, based on the qubit coords defined in stim_circuit)
def circuitWithError(stim_circuit: stim.Circuit, err: str, idx: int) -> str:
    coords = list(stim_circuit.get_final_qubit_coordinates().items())
    
    # Sort qubits by coords in circuit line order (bottom to top on viz, for easy slider use)
    coords.sort(key=lambda pair: (-pair[1][1], -pair[1][0]))
    
    return "" if err == "none" else f"#!pragma MARK{err}(0) {coords[min(idx, len(coords) - 1)][0]}\n" + str(stim_circuit)

# Update a slider's max value and color based on given Pauli err and num_lines in the circuit
def updatePauliSlider(slider_w_buttons: SliderWithButtons, err: str, num_lines: int) -> None:
    legend = {
        "X":"red",
        "Y":"green",
        "Z":"blue",
        "none":"black"
    }
    slider_w_buttons.slider.style.handle_color = legend[err]

    slider_w_buttons.slider.max = num_lines - 1

# Returns a 2x2 rotated surface code (as a stim str), with the given Pauli err on the idx-th circuit line (from bottom)
def updateErrorCircuit(slider_w_buttons: SliderWithButtons, err: str, idx: int) -> str:
    stim_circuit = stimSurfaceCode(True, 2, 2)
    updatePauliSlider(slider_w_buttons, err, len(stim_circuit.get_final_qubit_coordinates()))
    return circuitWithError(stim_circuit, err, idx)

# Create error config widgets + circuit widget
pauli_slider = PauliSlider()
pauli_select = PauliSelect()
circuit_with_error = CircuitWidget()

# Link error slider and selector to circuit
circuit_with_error_callback = lambda _: updateErrorCircuit(pauli_slider, pauli_select.value, pauli_slider.value)
ipywidgets.dlink((pauli_select, "value"), (circuit_with_error, "stim"), circuit_with_error_callback)
ipywidgets.dlink((pauli_slider, "value"), (circuit_with_error, "stim"), circuit_with_error_callback)

ipywidgets.HBox([circuit_with_error, pauli_slider, pauli_select])

## Exploring Surface Codes

Combining what we've done in the previous sections, we can create a single interactive output with planar code controls *and* Pauli error controls:

In [10]:
# Creates a planar code memory experiment circuit with the given is_rotated, width, and height parameters
# that has the given Pauli err marker on the idx-th circuit line (from bottom)
def finalCircuit(slider_w_buttons: SliderWithButtons, is_rotated: bool, w: int, h: int, err: str, idx: int) -> str:
    stim_circuit = stimSurfaceCode(is_rotated, w, h)
    updatePauliSlider(slider_w_buttons, err, len(stim_circuit.get_final_qubit_coordinates()))
    return circuitWithError(stim_circuit, err, idx)

# Planar code config widgets + pauli error config widgets + a circuit widget
final_circuit = CircuitWidget()
final_pauli_slider = PauliSlider()
final_pauli_select = PauliSelect()
final_w_slider = WSlider()
final_h_slider = HSlider()
final_checkbox = Checkbox()

# Link all of the config widgets to the circuit widget
final_callback = lambda _: finalCircuit(slider_w_buttons=final_pauli_slider, is_rotated=final_checkbox.value, w=final_w_slider.value, h=final_h_slider.value, err=final_pauli_select.value, idx=final_pauli_slider.value)
ipywidgets.dlink((final_pauli_select, "value"), (final_circuit, "stim"), final_callback)
ipywidgets.dlink((final_pauli_slider, "value"), (final_circuit, "stim"), final_callback)
ipywidgets.dlink((final_w_slider, "value"), (final_circuit, "stim"), final_callback)
ipywidgets.dlink((final_h_slider, "value"), (final_circuit, "stim"), final_callback)
ipywidgets.dlink((final_checkbox, "value"), (final_circuit, "stim"), final_callback)

display(ipywidgets.Label("Planar code settings:"), final_checkbox, final_w_slider, final_h_slider, ipywidgets.HBox([final_circuit, final_pauli_slider, final_pauli_select]))

Label(value='Planar code settings:')

Checkbox(value=True, description='Rotated?')

SliderWithButtons(children=(HBox(children=(Button(description='–', layout=Layout(width='30px'), style=ButtonSt…

SliderWithButtons(children=(HBox(children=(Button(description='–', layout=Layout(width='30px'), style=ButtonSt…

### Interactivity with `plotly`

Let's explore another interactive visualization. `draw_patch` uses `plotly` to visualize a patch of a given surface code, similar to the static visualization at the beginning of this notebook. Clicking on qubits on the interactive plot will also cycle between Pauli errors on that qubits (none → Pauli X (red) → Pauli Y (green) → Pauli Z (blue)):

In [11]:
rp_code = RotatedPlanarCode(width=5, height=5)
draw_patch(rp_code)

FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': '#d3d1c0',
              'hovertemplate': 'X Stabilizer<extra></extra>',
              'line': {'color': '#d3d1c0', 'width': 1},
              'mode': 'lines',
              'name': 'X Stabilizer',
              'showlegend': False,
              'type': 'scatter',
              'uid': '6bdd2922-e247-4be6-a382-f147736e0c92',
              'x': [9, 9, 10, 9],
              'y': [9, 7, 8, 9]},
             {'fill': 'toself',
              'fillcolor': '#d3d1c0',
              'hovertemplate': 'X Stabilizer<extra></extra>',
              'line': {'color': '#d3d1c0', 'width': 1},
              'mode': 'lines',
              'name': 'X Stabilizer',
              'showlegend': False,
              'type': 'scatter',
              'uid': '8d8366dc-a386-4801-8852-bb861ec2e989',
              'x': [9, 9, 10, 9],
              'y': [5, 3, 4, 5]},
             {'fill': 'toself',
              'fillcolor': '#3ccbda',
       

By linking this widget with a `CircuitWidget`, we can create an clickable surface code patch with a linked circuit visualization. Clicking a qubit on the interactive graph changes the Pauli error present at that qubit on the circuit diagram (with errors automatically propagating through the circuit):

In [12]:
# Returns the qubit at the specifed qubit coord in stim_circuit (-1 if not found)
def lookupQubitFromCoord(stim_circuit: stim.Circuit, target_coord: list[int, int]) -> int:
  coords = list(stim_circuit.get_final_qubit_coordinates().items())
  for idx, coord in coords:
    if coord == target_coord:
      return idx
  return -1

# Takes a map of qubit coords to Pauli errors and applies them at the start of the given stim circuit.
# Returns the resulting stim circuit (str).
def circuitWithPaulisAt(stim_circuit: stim.Circuit, active_paulis: dict[tuple[int, int], str]) -> str:
  stim_str = str(stim_circuit)

  err_circuit = "".join([
    f"#!pragma MARK{pauli.upper()}(0) {lookupQubitFromCoord(stim_circuit, list(coord))}\n" for coord, pauli in active_paulis.items()
  ]) + stim_str
  return err_circuit

# Given a circuit, CircuitWidget, dict of active paulis, and a new point, cycle the Pauli error at that point (X->Y->Z->none->X->...).
# Updates the given circuit_widg's .stim to reflect original_circuit with the updated Paulis.
def updateCicruitPaulis(points: Points, original_circuit: stim.Circuit, circuit_widg: CircuitWidget, active_paulis: dict[tuple[int, int], str]) -> None:
  xs = points.xs
  ys = points.ys

  if not xs or not ys or len(xs) != len(ys):
    return
    
  for point in zip(xs, ys):
    if point in active_paulis:
      if active_paulis[point] == "Z": # Z->none
        del active_paulis[point]
      else:
        active_paulis[point] = chr(ord(active_paulis[point]) + 1)
    else: # none->X
      active_paulis[point] = "X"

  circuit_widg.stim = circuitWithPaulisAt(original_circuit, active_paulis)

# An planar code visualizer widget containing a plotly and crumpy visualization
class InteractivePlanarCode(ipywidgets.HBox):
    def __init__(self, planar_code: RotatedPlanarCode | UnrotatedPlanarCode):
        super().__init__(
            layout=ipywidgets.Layout(
                justify_content="center",
                align_items="flex-start",
                width="auto",
            )
        )
        self.plot_widg = None
        self.circuit_widg = None

        self.set_planar_code(planar_code)

    def set_planar_code(self, planar_code: RotatedPlanarCode | UnrotatedPlanarCode):
        # Create planar code memory circuit
        deltakit_circuit = css_code_memory_circuit(
            css_code=planar_code,
            num_rounds=1,
            logical_basis=PauliBasis.Z,
        )

        # Strip detectors/observables for cleaner circuit visual
        deltakit_circuit = Circuit([
            layer for layer in deltakit_circuit.layers 
            if isinstance(layer, GateLayer)
        ])

        # Create CircuitWidget from the circuit
        stim_circuit = deltakit_circuit.as_stim_circuit()
        if self.circuit_widg is None:
            self.circuit_widg = CircuitWidget.from_stim(stim_circuit)
            self.circuit_widg.layout.max_height = "650px"
            self.circuit_widg.layout.max_width = "650px"
            self.circuit_widg.layout.overflow = "auto"
        else:
            self.circuit_widg.stim = str(stim_circuit)

        # Register click callback
        active_paulis = {}
        def qubitClickCallback(points: Points):
            updateCicruitPaulis(points, stim_circuit, self.circuit_widg, active_paulis)

        # Draw interactive patch
        unrotated_code = isinstance(planar_code, UnrotatedPlanarCode)
        if self.plot_widg is None:
            self.plot_widg = draw_patch(
                planar_code,
                qubitClickCallback,
                qubitClickCallback,
                unrotated_code=unrotated_code,
            )
        else:
            update_patch(
                planar_code,
                self.plot_widg,
                qubitClickCallback,
                qubitClickCallback,
                unrotated_code=unrotated_code,
            )

        # Replace children
        if not self.children:
            self.children = [self.plot_widg, self.circuit_widg]

ipc = InteractivePlanarCode(getPlanarCode(True, 3, 3))
ipc

InteractivePlanarCode(children=(FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': '#d3…

Finally, we can add the width slider, height slider, and rotation checkbox used previously to vary the planar code parameters:

In [13]:
# Planar code config + interactive planar code widgets
w_slider_interact = WSlider()
h_slider_interact = HSlider()
checkbox_interact = Checkbox()
ipc_with_sliders = InteractivePlanarCode(getPlanarCode(True, 2, 2))

# Change events only fire on release of slider
w_slider_interact.slider.continuous_update = False
h_slider_interact.slider.continuous_update = False

# Register callbacks on config sliders that update the current planar code viz
interact_callback = lambda _=None: ipc_with_sliders.set_planar_code(getPlanarCode(is_rotated=checkbox_interact.value, w=w_slider_interact.value, h=h_slider_interact.value))
w_slider_interact.observe(interact_callback, names="value")
h_slider_interact.observe(interact_callback, names="value")
checkbox_interact.observe(interact_callback, names="value")

interact_callback()

ipywidgets.VBox([checkbox_interact, w_slider_interact, h_slider_interact, ipc_with_sliders], layout=ipywidgets.Layout(align_items="center"))